In [31]:
import json
import os
import random
import time
import requests
import pandas as pd
from datetime import datetime


In [4]:
BASE_URL = 'https://api.coingecko.com/api/v3/'

def simple_request(req):
    api_url = BASE_URL + req
    raw = requests.get(api_url).json()
    return raw 

In [37]:
end_date = datetime.today()

datelist = pd.date_range(end=end_date, periods=5).strftime("%d-%m-%Y").tolist()
datelist

['08-12-2021', '09-12-2021', '10-12-2021', '11-12-2021', '12-12-2021']

In [26]:
res = simple_request('/coins/dogecoin/history?date=05-05-2021')

In [1]:
import saspy, os
print(saspy.__file__.replace('__init__.py', 'sascfg_personal.py'))

/home/vlad/.miniconda3/lib/python3.7/site-packages/saspy/sascfg_personal.py


In [24]:
res = {}
for date in datelist:
    simple_request('/coins/dogecoin/history?date={}'.format(date))

{'id': 'dogecoin', 'symbol': 'doge', 'name': 'Dogecoin', 'localization': {'en': 'Dogecoin', 'de': 'Dogecoin', 'es': 'Dogecoin', 'fr': 'Dogecoin', 'it': 'Dogecoin', 'pl': 'Dogecoin', 'ro': 'Dogecoin', 'hu': 'Dogecoin', 'nl': 'Dogecoin', 'pt': 'Dogecoin', 'sv': 'Dogecoin', 'vi': 'Dogecoin', 'tr': 'Dogecoin', 'ru': 'Dogecoin', 'ja': 'ドージコイン', 'zh': '狗狗币', 'zh-tw': '狗狗幣', 'ko': '도지코인', 'ar': 'الدوجكوين', 'th': 'Dogecoin', 'id': 'Dogecoin'}, 'image': {'thumb': 'https://assets.coingecko.com/coins/images/5/thumb/dogecoin.png?1547792256', 'small': 'https://assets.coingecko.com/coins/images/5/small/dogecoin.png?1547792256'}, 'market_data': {'current_price': {'aed': 0.7896326369105757, 'ars': 21.700231405277346, 'aud': 0.30185700262186704, 'bch': 0.00037696225741313123, 'bdt': 18.444907112986886, 'bhd': 0.0810510022594369, 'bmd': 0.21498255296764252, 'bnb': 0.0003445393385855585, 'brl': 1.2091476559286551, 'btc': 3.772092271453011e-06, 'cad': 0.274755227082001, 'chf': 0.19765904386695712, 'clp':